<a href="https://colab.research.google.com/github/IdjiotSandwiches/knn-fer/blob/create-model/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dagshub --quiet
!pip install mlflow --quiet

In [ ]:
import cv2 as cv
import numpy as np
import mlflow
import dagshub
import os
import pathlib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
PATH = pathlib.Path('facial-emotion-recognition-augmented')
LABELS = os.listdir(PATH)

In [ ]:
dagshub.init(repo_owner='IdjiotSandwiches', repo_name='knn-fer', mlflow=True)

Accessing as IdjiotSandwiches

Initialized MLflow to track repo "IdjiotSandwiches/knn-fer"

Repository IdjiotSandwiches/knn-fer initialized!

In [ ]:
K_SIZE = (31,31)
SIGMA = 4
THETA_RANGE = np.arange(0, np.pi, np.pi/32)
LAMBD = 10.0
GAMMA = 0.5
PSI = 0
FLOATING_POINT = cv.CV_64F

gabor_params = {
    'ksize': K_SIZE,
    'sigma': SIGMA,
    'lambd': LAMBD,
    'gamma': GAMMA,
    'psi': PSI
}

KERNELS = [cv.getGaborKernel(**gabor_params, theta=theta) for theta in THETA_RANGE]

In [ ]:
def open_dataset(dir):
  images, labels = [], []

  for i, label in enumerate(LABELS):
    path = f'{dir}/{label}'
    for j, img_path in enumerate(os.listdir(path)):
      img = cv.imread(f'{path}/{img_path}', cv.IMREAD_GRAYSCALE)
      img = cv.resize(img, (48,48))
      img = cv.GaussianBlur(img,(5,5),0)
      img = cv.equalizeHist(img)
      img = img / 255.0
      img = gabor_filter(img)
      images.append(img)
      labels.append(i)

  return np.array(images), np.array(labels)

In [ ]:
def gabor_filter(img):
  return [cv.filter2D(img, FLOATING_POINT, kernel) for kernel in KERNELS]

In [ ]:
def save_filtered_img(images, labels):
  folder_path = f'/content/drive/MyDrive/gabor-filtered-imgs'
  os.makedirs(folder_path, exist_ok=True)

  np.save(f'{folder_path}/images.npy', images)
  np.save(f'{folder_path}/labels.npy', labels)

  print('Ok!')

In [ ]:
images, labels = open_dataset(PATH)
images = images.reshape(images.shape[0], -1)
save_filtered_img(images, labels)

Ok!


In [ ]:
images = np.load('/content/drive/MyDrive/gabor-filtered-imgs/images.npy', mmap_mode='r')
labels = np.load('/content/drive/MyDrive/gabor-filtered-imgs/labels.npy', mmap_mode='r')

In [ ]:
images = np.load('/content/drive/MyDrive/gabor-filtered-imgs/images.npy', mmap_mode='r')
labels = np.load('/content/drive/MyDrive/gabor-filtered-imgs/labels.npy', mmap_mode='r')

N_NEIGHBORS = 9
WEIGHTS = 'distance'
METRIC = 'manhattan'
ALGORITHM = 'ball_tree'
TEST_SIZE = 0.7
RANDOM_STATE = 42
N_SPLITS = 8
SHUFFLE = True

model_params = {
  'n_neighbors': N_NEIGHBORS,
  'weights': WEIGHTS,
  'metric': METRIC,
  'algorithm': ALGORITHM
}

calib_params = {
  'test_size': TEST_SIZE,
  'random_state': RANDOM_STATE
}

kfold_params = {
    'n_splits': N_SPLITS,
    'shuffle': SHUFFLE
}

skf = StratifiedKFold(**kfold_params)
i = 10

for train_idx, test_idx in skf.split(images, labels):
  X_train, X_test = images[train_idx], images[test_idx]
  y_train, y_test = labels[train_idx], labels[test_idx]

  X_calib, X_test, y_calib, y_test = train_test_split(X_test, y_test, **calib_params)

  model = KNeighborsClassifier(**model_params)
  model.fit(X_train, y_train)

  calib_model = CalibratedClassifierCV(model, cv="prefit")
  calib_model.fit(X_calib, y_calib)

  predict = calib_model.predict(X_test)

  metrics = {
    'accuracy': accuracy_score(y_test, predict),
    'precision': precision_score(y_test, predict, average='macro'),
    'recall': recall_score(y_test, predict, average='macro'),
    'f1': f1_score(y_test, predict, average='macro'),
  }

  print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}")
  print(classification_report(y_test, predict))

  mlflow.set_experiment("KNN_Model Augmented StratifiedKFold Calibrated")
  mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow")

  with mlflow.start_run(run_name=f'KNN StratifiedKFold={i}'):
    mlflow.log_params(gabor_params)
    mlflow.log_params(calib_params)
    mlflow.log_params(model_params)
    mlflow.log_params(kfold_params)
    mlflow.log_param('floating_point', FLOATING_POINT)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(
        sk_model=calib_model,
        artifact_path='KNN Model',
        input_example=X_train[:1]
    )
  i = i + 1

Accuracy: 0.5487804878048781
Precision: 0.555321708386144
Recall: 0.5535319036103336
F1 Score: 0.5494977723701872
              precision    recall  f1-score   support

           0       0.58      0.68      0.63        66
           1       0.59      0.48      0.53        75
           2       0.63      0.62      0.62        65
           3       0.40      0.54      0.46        61
           4       0.65      0.69      0.67        71
           5       0.47      0.47      0.47        81
           6       0.56      0.40      0.46        73

    accuracy                           0.55       492
   macro avg       0.56      0.55      0.55       492
weighted avg       0.56      0.55      0.55       492



2024/12/05 15:42:39 INFO mlflow.tracking.fluent: Experiment with name 'KNN_Model Augmented StratifiedKFold Calibrated' does not exist. Creating a new experiment.


## **Tuning**

In [ ]:
N_NEIGHBORS = 9
WEIGHTS = 'distance'
METRIC = 'manhattan'
ALGORITHM = 'ball_tree'

TEST_SIZE = 0.3
CALIB_SIZE = 0.5
RANDOM_STATE = 42

split_params = {
    'test_size': TEST_SIZE,
    'random_state': RANDOM_STATE
}

calib_params = {
    'test_size': CALIB_SIZE,
    'random_state': RANDOM_STATE
}

X_train, X_test, y_train, y_test = train_test_split(images, labels, **split_params)
X_test, X_calib, y_test, y_calib = train_test_split(X_test, y_test, **calib_params)

model_params = {
  'n_neighbors': N_NEIGHBORS,
  'weights': WEIGHTS,
  'metric': METRIC,
  'algorithm': ALGORITHM
}

model = KNeighborsClassifier(**model_params)
model.fit(X_train, y_train)

calib_model = CalibratedClassifierCV(model, cv="prefit")
calib_model.fit(X_calib, y_calib)

predict = calib_model.predict(X_test)

metrics = {
  'accuracy': accuracy_score(y_test, predict),
  'precision': precision_score(y_test, predict, average='macro'),
  'recall': recall_score(y_test, predict, average='macro'),
  'f1': f1_score(y_test, predict, average='macro'),
}

print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}")
print(classification_report(y_test, predict))

mlflow.set_experiment("KNN_Model using calibration")
mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow")

with mlflow.start_run(run_name=f'KNN sigma=5'):
  mlflow.log_params(gabor_params)
  mlflow.log_params(split_params)
  mlflow.log_param('calib_test_size', calib_params['test_size'])
  mlflow.log_param('calib_random_state', calib_params['random_state'])
  mlflow.log_params(model_params)
  mlflow.log_metrics(metrics)
  mlflow.sklearn.log_model(
      sk_model=calib_model,
      artifact_path='KNN Model',
      input_example=X_train[:1]
  )

Accuracy: 0.41271820448877805
Precision: 0.35505141285443115
Recall: 0.33373441164794554
F1 Score: 0.3324115318106068


2024/12/04 03:16:05 INFO mlflow.tracking.fluent: Experiment with name 'KNN_Model using calibration' does not exist. Creating a new experiment.


🏃 View run KNN sigma=5 at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/8/runs/c1b2dcc5917e455fa84e0b57000e5062
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/8


RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

In [ ]:
for p, y in zip(predict[:20], y_test[:20]):
  print(f'Predict: {LABELS[p]} | Real: {LABELS[y]}')

Predict: happy | Real: surprise
Predict: surprise | Real: surprise
Predict: surprise | Real: fear
Predict: happy | Real: happy
Predict: happy | Real: sad
Predict: surprise | Real: neutral
Predict: neutral | Real: sad
Predict: happy | Real: happy
Predict: angry | Real: neutral
Predict: sad | Real: happy
Predict: happy | Real: angry
Predict: sad | Real: sad
Predict: happy | Real: angry
Predict: sad | Real: angry
Predict: happy | Real: angry
Predict: disgust | Real: angry
Predict: angry | Real: angry
Predict: surprise | Real: fear
Predict: surprise | Real: surprise
Predict: angry | Real: fear


In [ ]:
# N_NEIGHBORS = [3,5]
# WEIGHTS = ['uniform', 'distance']
# METRIC = ['minkowski', 'euclidean', 'manhattan']
# ALGORITHM = ['auto', 'ball_tree', 'kd_tree', 'brute']

# for n in N_NEIGHBORS:
#   for w in WEIGHTS:
#     for m in METRIC:
#       for a in ALGORITHM:
#         model_params = {
#             'n_neighbors': n,
#             'weights': w,
#             'metric': m,
#             'algorithm': a
#         }

#         model = KNeighborsClassifier(**model_params)
#         model.fit(X_train, y_train)

#         predict = model.predict(X_test)

#         metrics = {
#           'accuracy': accuracy_score(y_test, predict),
#           'precision': precision_score(y_test, predict, average='macro'),
#           'recall': recall_score(y_test, predict, average='macro'),
#           'f1': f1_score(y_test, predict, average='macro')
#         }

#         print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}")

#         mlflow.set_experiment("KNN_Model Theta=32")
#         mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow")

#         with mlflow.start_run(run_name=f'KNN n_neighbors={n}, weights={w}, metric={m}, algorithm={a}'):
#           mlflow.log_params(gabor_params)
#           mlflow.log_params(split_params)
#           mlflow.log_params(model_params)
#           mlflow.log_metrics(metrics)
#           mlflow.sklearn.log_model(
#               sk_model=model,
#               artifact_path='KNN Model',
#               input_example=X_train[:1]
#           )

Accuracy: 0.3074766355140187
Precision: 0.3050938501361262
Recall: 0.29387967874509396
F1 Score: 0.2783373964588805


2024/11/30 10:59:14 INFO mlflow.tracking.fluent: Experiment with name 'KNN_Model Theta=32' does not exist. Creating a new experiment.


🏃 View run KNN n_neighbors=3, weights=uniform, metric=minkowski, algorithm=auto at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/b02f353aaeb3491cb74358282af9a0b4
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.308411214953271
Precision: 0.30554388973180335
Recall: 0.2947200148795478
F1 Score: 0.27900485072077663


🏃 View run KNN n_neighbors=3, weights=uniform, metric=minkowski, algorithm=ball_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/923932c2679642a08150dd7eb2980d57
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.308411214953271
Precision: 0.30564408809920407
Recall: 0.2947200148795478
F1 Score: 0.2790072898045715
🏃 View run KNN n_neighbors=3, weights=uniform, metric=minkowski, algorithm=kd_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/aa15c822146048458410e873536155a7
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.3074766355140187
Precision: 0.3050938501361262
Recall: 0.29387967874509396
F1 Score: 0.2783373964588805
🏃 View run KNN n_neighbors=3, weights=uniform, metric=minkowski, algorithm=brute at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/df424c2fee9e46bd9b4a31f40bc56106
🧪 View exp

Accuracy: 0.308411214953271
Precision: 0.30554388973180335
Recall: 0.2947200148795478
F1 Score: 0.27900485072077663


🏃 View run KNN n_neighbors=3, weights=uniform, metric=euclidean, algorithm=ball_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/eec782be7df14b27a88a49477edf3286
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.308411214953271
Precision: 0.30564408809920407
Recall: 0.2947200148795478
F1 Score: 0.2790072898045715
🏃 View run KNN n_neighbors=3, weights=uniform, metric=euclidean, algorithm=kd_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/f6a2dd52f9ad4de7928c6899918adb7c
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.3074766355140187
Precision: 0.3050938501361262
Recall: 0.29387967874509396
F1 Score: 0.2783373964588805


🏃 View run KNN n_neighbors=3, weights=uniform, metric=euclidean, algorithm=brute at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/7da638b7f90b4ffc924ca1bba56e99b3
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.3233644859813084
Precision: 0.31787422780360425
Recall: 0.30837462993039805
F1 Score: 0.2955184378198163
🏃 View run KNN n_neighbors=3, weights=uniform, metric=manhattan, algorithm=auto at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/c0fe6cc1f71b4d0890cca4f4d474bb7d
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.3233644859813084
Precision: 0.3177394374501707
Recall: 0.30837462993039805
F1 Score: 0.2954523945289038
🏃 View run KNN n_neighbors=3, weights=uniform, metric=manhattan, algorithm=ball_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/3b8d1d02eaf148a78fbb3c667b6a18ba
🧪 View expe

🏃 View run KNN n_neighbors=3, weights=distance, metric=minkowski, algorithm=ball_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/f055b28fb9714f9fb135bc5babb520f8
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.34766355140186916
Precision: 0.3287131082820621
Recall: 0.3130277230955814
F1 Score: 0.31473980038813476
🏃 View run KNN n_neighbors=3, weights=distance, metric=minkowski, algorithm=kd_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/149874b20f7a402a9ec2acd812da0472
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.34579439252336447
Precision: 0.3263228137046797
Recall: 0.3106175282484117
F1 Score: 0.3123337182888574
🏃 View run KNN n_neighbors=3, weights=distance, metric=minkowski, algorithm=brute at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/dfff4fde858b47119d34d5c534636329
🧪 Vie

Accuracy: 0.34766355140186916
Precision: 0.3286418668785575
Recall: 0.3130277230955814
F1 Score: 0.3147611975894672
🏃 View run KNN n_neighbors=3, weights=distance, metric=euclidean, algorithm=ball_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/42485c29c4e0471089b6564a796b58d6
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4


Accuracy: 0.34766355140186916
Precision: 0.3287131082820621
Recall: 0.3130277230955814
F1 Score: 0.31473980038813476
🏃 View run KNN n_neighbors=3, weights=distance, metric=euclidean, algorithm=kd_tree at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/344be1df03114619a9efdab34917c339
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.34579439252336447
Precision: 0.3263228137046797
Recall: 0.3106175282484117
F1 Score: 0.3123337182888574
🏃 View run KNN n_neighbors=3, weights=distance, metric=euclidean, algorithm=brute at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4/runs/32696a22a213424999b7f229720bcf01
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow/#/experiments/4
Accuracy: 0.3663551401869159
Precision: 0.3502625307532238
Recall: 0.3333851627875379
F1 Score: 0.33528941745796753
🏃 View run KNN n_neighbors=3, weights=distance, metric=manhattan, algorithm=auto at: 